**Partie 1**

$1.1 Jeu \ de \ données$

$1-$ Les données d'apprentissages servent à entraîner le modèle. Tandis que les données de validation utilisées au cours de l'entrainement, elles permettent de régler les hyper paramètres du modèle. Quant aux données de test, elles permettent de s'assurer de la capacité du modèle à pouvoir généraliser ses prédictions sur de nouvelles données.

$2-$ Plus N est grand, plus le modèle apprend à mieux généraliser et évite l'overfitting. Cela peut cependant nécessiter une plus rande puissance de calcul au niveau du matériel et un temps d'entrainement plus long.

$3-$ L'utilisation des fonctions d'activation permet d'une part d'introduire la non linéarité (pour les fonctions d'activation non linéaires) et d'apprendre des relations non linéaire complexes, d'autre part 

$4-$ nx : représente le nombre de caractéristiques d'un individu de l'échantillon, nh: le nombre de neurones dans la couche cachée, ny : le nombre de sortie. En pratique, nx est choisi en fonction du nombre de caractéristiques présentes chez un individu du dataset, ny est choisi est choisi en fonction du nombre de classes à classifier, nh de manière aléatoire.

$5-$ yˆ représente les prédictions du modèle. Par contre y lui représente les vraies valeurs.

$6-$ On utilise une fonction softmax pour normaliser les sorties de chaque neurone présent dans la couche de sortie afin d'avoir une somme des probabilités égale à 1.

$7-$ <center width="100%"><img src="./img/q7.jpeg" width="300px"></center>

$8-$ Pour l'entropie croisé, les yˆi doivent tendrent vers 1. Quant à l'erreur quadratique, les yˆi doivent s'approcher le plus des yi

$9-$ L’erreur quadratique moyenne (EQM) est un indicateur de vérification de la fiabilité d’un modèle. Il permet de mesurer les écarts entre les valeurs réellement observées et les valeurs prédites par le modèle. Quant à l'entropie croisé, il permet de mesurer de façon précise si les classes réellement observées et les classes prédites par le modèle correspondent.

$10-$

$11-$

$12-$

$13-$

$14-$

$15-$

$16-$

$17-$

**Partie 2**

In [1]:
import torch
import torch.nn as nn
from tqdm.notebook import tqdm
from statistics import mean
import matplotlib.pyplot as plt
import numpy as np

In [117]:
def init_params(nx, nh, ny):
    params = {}
    wh = torch.normal(0., 0.3, size=(nh, nx))
    bh = torch.normal(0., 0.3, size=(nh, 1))
    wy = torch.normal(0., 0.3, size=(nh, ny))
    by = torch.normal(0., 0.3, size=(ny, 1))
    params["wh"] = wh
    params["bh"] = bh
    params["wy"] = wy
    params["by"] = by
    return params

# Test part

x = init_params(2, 2, 2)
print(x)


{'wh': tensor([[ 0.0776,  0.6557],
        [ 0.3593, -0.0125]]), 'bh': tensor([[0.5023],
        [0.2037]]), 'wy': tensor([[0.1396, 0.0859],
        [0.0868, 0.0430]]), 'by': tensor([[-0.1898],
        [-0.2706]])}


In [118]:
Xx = torch.tensor([[1., 1.],
                  [1., 1.],
                  [1., 1.]])
for k in x:
    x[k].requires_grad_(True)

In [119]:
def forward(params, X):
    output = {}
    #X.requires_grad_(True)
    htild = X @ torch.transpose(params["wh"], 0, 1) + torch.transpose(params["bh"], 0, 1)
    h = torch.tanh(htild)
    ytild = h @ params["wy"] + torch.transpose(params["by"], 0, 1)
    ythat = torch.softmax(ytild, dim=0)
    #output["X"] = X
    output["htild"] = htild
    output["h"] = h
    output["ytild"] = ytild
    output["ythat"] = ythat
    return output

# Test part

a = forward(x, Xx)
print(a)


{'htild': tensor([[1.2356, 0.5506],
        [1.2356, 0.5506],
        [1.2356, 0.5506]], grad_fn=<AddBackward0>), 'h': tensor([[0.8442, 0.5009],
        [0.8442, 0.5009],
        [0.8442, 0.5009]], grad_fn=<TanhBackward0>), 'ytild': tensor([[-0.0285, -0.1765],
        [-0.0285, -0.1765],
        [-0.0285, -0.1765]], grad_fn=<AddBackward0>), 'ythat': tensor([[0.3333, 0.3333],
        [0.3333, 0.3333],
        [0.3333, 0.3333]], grad_fn=<SoftmaxBackward0>)}


In [120]:
def loss_accuracy(Yhat, Y):
    _, ind = torch.max(Yhat, dim=1)
    loss = -1 * torch.sum(_ * Y) + torch.log(torch.sum(torch.exp(_)))
    acc = 0
    for k in range(len(Y)):
        if ind[k] == Y[k]:
            acc = acc + 1
    return {
        "L": loss,
        "acc": acc / len(Y),
    }

# Test part
"""
yhat = torch.tensor([[0.333, 0.6333],
        [0.3333, 0.3333],
        [0.6333, 0.3333]])
y = torch.tensor([1, 0, 1])
loss_accuracy(yhat, y)
"""
los = loss_accuracy(a["ythat"], y)

In [121]:
los

{'L': tensor(0.7653, grad_fn=<AddBackward0>), 'acc': 0.3333333333333333}

In [122]:
los['L'].backward()

In [124]:
x["bh"].grad

tensor([[0.],
        [0.]])

In [134]:
def backward(params, outputs, Y):
    """
    set all grads to None before calling this function
    params : are weights
    outputs : list that contains the gradients of loss of every batch
    Y : equivalent to L (loss) which is the output of loss_accuracy
    """
    #nn.Module.zero_grad() : set all grads to None here
    Y.backward()
    outputs.append(params.grad)

# Test part
"""
x = torch.tensor([1], dtype=torch.float)
x.requires_grad_(True)
loss = x @ x
output = []
backward(x, output, loss)
output
print(x)
print(output)
"""

'\nx = torch.tensor([1], dtype=torch.float)\nx.requires_grad_(True)\nloss = x @ x\noutput = []\nbackward(x, output, loss)\noutput\nprint(x)\nprint(output)\n'

In [136]:
def sgd(params, grads, eta):
    """
    This function will return the updated params
    params : are weights
    eta : pas d'apprentissage
    grads : List of gradients
    """
    with torch.no_grad():
        #params = params - eta * grads[-1]
        params = torch.sub(params, grads[-1], alpha=eta)
    return params.requires_grad_(True)

# Test part
"""
x = torch.tensor([1], dtype=torch.float)
x.requires_grad_(True)
loss = x @ x
output = []
backward(x, output, loss)
print("x = ", x)
print("grads = ", output[-1])
x = sgd(x, output, 0.01)
print("updated x = ", x)
"""

'\nx = torch.tensor([1], dtype=torch.float)\nx.requires_grad_(True)\nloss = x @ x\noutput = []\nbackward(x, output, loss)\nprint("x = ", x)\nprint("grads = ", output[-1])\nx = sgd(x, output, 0.01)\nprint("updated x = ", x)\n'

In [174]:
class TP1_Model(nn.Module):
    def __init__(self, nx, nh, ny):
        super().__init__()
        self.nx = nx
        self.nh = nh
        self.ny = ny
        self.params = self.init_params(nx, nh, ny)

    def init_params(self, nx, nh, ny):
        params = {}
        wh = torch.normal(0., 0.3, size=(nh, nx))
        bh = torch.normal(0., 0.3, size=(nh, 1))
        wy = torch.normal(0., 0.3, size=(nh, ny))
        by = torch.normal(0., 0.3, size=(ny, 1))
        params["wh"] = wh
        params["bh"] = bh
        params["wy"] = wy
        params["by"] = by
        return params

    def forward(self, params, X):
        pass
    def loss_accuracy(Yhat, Y):
        pass
    def backward(params, outputs, Y):
        """
        set all grads to None before calling this function
        params : are weights
        outputs : list that contains the gradients of loss of every batch
        Y : equivalent to L (loss) which is the output of loss_accuracy
        """
        #nn.Module.zero_grad() : set all grads to None here
        Y.backward()
        outputs.append(params.grad)
    def sgd(params, grads, eta):
        """
        This function will return the updated params
        params : are weights
        eta : pas d'apprentissage
        grads : List of gradients
        """
        with torch.no_grad():
            #params = params - eta * grads[-1]
            params = torch.sub(params, grads[-1], alpha=eta)
        return params.requires_grad_(True)

In [145]:
def train_model(model, data_loader, num_epochs=100, learning_rate=0.01):
    
    history = {"loss": [],
              "acc": [],
              }
    # Training loop
    for epoch in tqdm(range(num_epochs)):
        outputs = []
        losses = []
        accs = []
        for data_inputs, data_labels in data_loader:
            
            ## Step 1: Run the model on the input data
            Yhat = model.forward(model.params, data_inputs)
            Yhat = preds.squeeze(dim=1) # Output is [Batch size, 1], but we want [Batch size]
            
            ## Step 2: Calculate the loss
            loss = loss_accuracy(Yhat, data_labels.float())
            losses.append(loss['L'])
            accs.append(loss["acc"])
            
            ## Step 3: Perform backpropagation
            model.zero_grad()
            model.backward(model.params, outputs, data_labels.float())
            
            ## Step 4: Update the parameters
            model.params = model.sgd(model.params, outputs, learning_rate)
        history["loss"].append(mean(losses))
        history["acc"].append(mean(accs))
    return history

In [153]:
def visualize_loss(history):
    x = np.arange(1, len(history["loss"]) + 1)
    plt.title("Courbe de loss")
    plt.xlabel("Iterations")
    plt.ylabel("Loss")
    plt.plot(x, history["loss"])
    plt.show()

In [159]:
def plot_data_with_grid():
    pass